libraries

In [1]:
import numpy as np
import pandas as pd
from keras_preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os

image properties

In [2]:
Image_Width=128
Image_Height=128
Image_Size=(Image_Width,Image_Height)
Image_Channels=3 #RGB Photos

Prepare datasets

In [3]:
filenames=os.listdir("./dataset/train")
categories=[]
for f_name in filenames:
    category=f_name.split('.')[0]
    if category=='dog':
        categories.append(1)
    else:
        categories.append(0)

df=pd.DataFrame({
    'filename':filenames,
    'category':categories
})

Neural network

In [4]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,\
     Dropout,Flatten,Dense,Activation,\
     BatchNormalization

model=Sequential()
# 3x3 boyutunda 32 adet filtreden oluşan ReLU aktivasyonlu CONV katmanı ekleyelim.
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(Image_Width,Image_Height,Image_Channels)))
model.add(BatchNormalization()) # Çıktıyı 0 ile 1 arasında normalize ediyor
# 2x2 boyutlu çerçeveden oluşan MAXPOOL katmanı ekleyelim. 
model.add(MaxPooling2D(pool_size=(2,2)))
# her seferinde nöronların %25'i atılsın (drop)
model.add(Dropout(0.25))

# 3x3 boyutunda 64 adet filtreden oluşan ReLU aktivasyonlu CONV katmanı ekleyelim. 
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# Tam bağlantılı (fully connected) katmanına geçiş olacağı için düzleştirme yapalım
model.add(Flatten())
# 128 nörondan oluşan ReLU aktivasyonu FC katmanı ekleyelim 
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
# Her seferinde %50'sini atalım (drop)
model.add(Dropout(0.5))
# Çıkış katmanına sınıf sayısı kadar (2) Softmax aktivasyonlu nöron ekleyelim
model.add(Dense(2,activation='softmax'))

# rmsprop optimizasyon yöntemini ve cross entropy yitim (loss) fonksiyonunu kullanalım.
model.compile(loss='categorical_crossentropy',
  optimizer='rmsprop',metrics=['accuracy'])

NOT 1) BatchNormalization(): Batch normalization sayesinde ağdaki katmanlar, önceki katmanın öğrenmesini beklemek zorunda kalmaz. Eş zamanlı olarak öğrenime olanak sağlar. Eğitimimizin hızlanmasını sağlar.

input->hidden layer->BatchNormalization()->hidden layer->BatchNormalization()->hidden layer->BatchNormalization()->output

Batch normalization kullanmadan yüksek bir learning rate kullansak, gradyanların yok olması problemiyle karşılaşabiliriz. Ancak batch norm ile bir katmandaki değişiklik diğer katmana yayılmadığı için daha yüksek learning rate’ler kullanabiliriz. Ayrıca batch norm, ağın daha kararlı ve düzenli hale gelmesini sağlar.

Analyzing model

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 126, 126, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 63, 63, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 batch_normalization_1 (Batc  (None, 61, 61, 64)       2

Learning rate

In [6]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystop = EarlyStopping(patience = 10) # Eğitimin durdurulacağı iyileşme olmayan epoch sayısı
# EarlyStopping : Eğitim esnasında takip ettiğiniz değerde belirleyeceğiniz adım — epoch — boyunca iyileşme yoksa eğitim verdiğiniz adım sayısından önce durduruluyor. 
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_accuracy',patience = 2,verbose = 1,factor = 0.5,min_lr = 0.00001)
#ReduceLROnPlateau : Eğitim esnasında takip ettiğiniz değerde belirleyeceğiniz adım — epoch — boyunca iyileşme yoksa Öğrenme hızını 
# sizin belirleyeceğiniz bir katsayıyla çarparak küçültüyor. Bizim kodumuzda 2 adım boyunca test verisi başarımında bir iyileşme olmuyorsa öğrenme hızı 0.5 çarpılarak yarıya indiriliyor.
callbacks = [earlystop,learning_rate_reduction]

Data info

In [7]:
df["category"] = df["category"].replace({0:'cat',1:'dog'})
train_df,validate_df = train_test_split(df,test_size=0.20,
  random_state=42)

train_df = train_df.reset_index(drop=True)# Hataya karşı indeksi sıfırlıyoruz
validate_df = validate_df.reset_index(drop=True)
total_train=train_df.shape[0]
total_validate=validate_df.shape[0]
batch_size=15
# batch_size => parametre güncellemesinin gerçekleştiği ağa verilen alt örneklerin sayısıdır. 
#Toplu boyut için iyi bir varsayılan değer 32 olabilir. Ayrıca 32, 64, 128, 256 ve benzeri değerleri de deneyin.

Train and val data_gen

In [8]:
train_datagen = ImageDataGenerator(rotation_range=15,
                                rescale=1./255,
                                shear_range=0.1,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1
                                )

train_generator = train_datagen.flow_from_dataframe(train_df,
                                                 "./dataset/train",x_col='filename',y_col='category',
                                                 target_size=Image_Size,
                                                 class_mode='categorical',
                                                 batch_size=batch_size)

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    "./dataset/train", 
    x_col='filename',
    y_col='category',
    target_size=Image_Size,
    class_mode='categorical',
    batch_size=batch_size
)

test_datagen = ImageDataGenerator(rotation_range=15,
                                rescale=1./255,
                                shear_range=0.1,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1)

test_generator = train_datagen.flow_from_dataframe(train_df,
                                                 "./dataset/train",x_col='filename',y_col='category',
                                                 target_size=Image_Size,
                                                 class_mode='categorical',
                                                 batch_size=batch_size)

Found 20000 validated image filenames belonging to 2 classes.
Found 5000 validated image filenames belonging to 2 classes.
Found 20000 validated image filenames belonging to 2 classes.


Model Training

In [9]:
epochs=10
history = model.fit(
    train_generator, 
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size, # (//) => mod işleminde kalanı verir.
    steps_per_epoch=total_train//batch_size,
    callbacks=callbacks
)

Epoch 1/10
1333/1333 [==============================] - 1085s 813ms/step - loss: 0.7475 - accuracy: 0.6378 - val_loss: 0.7872 - val_accuracy: 0.5808 - lr: 0.0010
Epoch 2/10
1333/1333 [==============================] - 1236s 928ms/step - loss: 0.5469 - accuracy: 0.7348 - val_loss: 0.4407 - val_accuracy: 0.7950 - lr: 0.0010
Epoch 3/10
1333/1333 [==============================] - 1301s 942ms/step - loss: 0.5056 - accuracy: 0.7626 - val_loss: 0.5486 - val_accuracy: 0.7411 - lr: 0.0010
Epoch 4/10
1333/1333 [==============================] - ETA: 0s - loss: 0.4727 - accuracy: 0.7804
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
1333/1333 [==============================] - 1138s 853ms/step - loss: 0.4727 - accuracy: 0.7804 - val_loss: 0.4775 - val_accuracy: 0.7796 - lr: 0.0010
Epoch 5/10
1333/1333 [==============================] - 1124s 843ms/step - loss: 0.4140 - accuracy: 0.8129 - val_loss: 0.3192 - val_accuracy: 0.8633 - lr: 5.0000e-04
Epoch 6/10
1333/1333 [=

callbacks = [earlystop,learning_rate_reduction]

Model save

**saving and loading the .h5 model**
 
**save model**
model.save('model1_catsVSdogs_10epoch.h5')
print('Model Saved!')
 
**load model**
savedModel=load_model('model1_catsVSdogs_10epoch.h5')
savedModel.summary()

In [10]:
model.save("model1_catsVSdogs_10epoch.h5")
print('Model Saved!')

Model Saved!


In [11]:
from keras.models import load_model
savedModel=load_model('model1_catsVSdogs_10epoch.h5')
savedModel.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 126, 126, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 63, 63, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 batch_normalization_1 (Batc  (None, 61, 61, 64)       2

Test data

In [12]:
test_filenames = os.listdir("./dataset/test1")
test_df = pd.DataFrame({
    'filename': test_filenames
})
nb_samples = test_df.shape[0]

Categorical prediction

In [13]:
predict = model.predict(test_generator, steps=np.ceil(nb_samples/batch_size))
# np.ceil => Girişin tavanını eleman bazında döndürün. x skalerinin tavanı en küçük tam sayıdır
# nb_samples = test_df.shape[0]

834/834 [==============================] - 157s 185ms/step


labels to categories

In [14]:
test_df['category'] = np.argmax(predict, axis=-1)

label_map = dict((v,k) for k,v in train_generator.class_indices.items())
test_df['category'] = test_df['category'].replace(label_map)

test_df['category'] = test_df['category'].replace({ 'dog': 1, 'cat': 0 })

prediction results

In [ ]:
sample_test = test_df.head(10)
sample_test.head()
plt.figure(figsize=(12, 24))
for index, row in sample_test.iterrows():
    filename = row['filename']
    category = row['category']
    img = load_img("./dataset/test1/"+filename, target_size=Image_Size)
    plt.subplot(6, 3, index+1)
    plt.imshow(img)
    plt.xlabel(filename + '(' + "{}".format(category) + ')' )
    plt.tight_layout()
    plt.show()

model performance

In [16]:
results={
    0:'cat',
    1:'dog'
}
from PIL import Image
import numpy as np
im=Image.open("./dataset/test1/10001.jpg")
im=im.resize(Image_Size)
im=np.expand_dims(im,axis=0)
im=np.array(im)
im=im/255

predict_x=model.predict([im])[0]
pred = np.argmax(predict_x,axis=-1)
print(results[pred])

1/1 [==============================] - 0s 254ms/step
cat


use program with tkinter!!!

In [ ]:
import tkinter as tk
from tkinter import filedialog
from tkinter import *
from PIL import ImageTk, Image
import numpy
from keras.models import load_model
model = load_model('cats&dogs_10_epoch.h5')
classes = { 
    0:'its a cat',
    1:'its a dog',
}
#GUI
top=tk.Tk()
top.geometry('800x600')
top.title('CatsVSDogs Classification')
top.configure(background='#CDCDCD')
label=Label(top,background='#CDCDCD', font=('arial',15,'bold'))
sign_image = Label(top)

def classify(file_path):
    global label_packed
    image = Image.open(file_path)
    image = image.resize((128,128))
    image = numpy.expand_dims(image, axis=0)
    image = numpy.array(image)
    image = image/255
    pred = model.predict([image])[0]
    pred = np.argmax(pred,axis=-1)
    sign = classes[pred]
    print(sign)
    label.configure(foreground='#011638', text=sign) 

def show_classify_button(file_path):
    classify_b=Button(top,text="Classify image",
   command=lambda: classify(file_path),
   padx=10,pady=5)
    classify_b.configure(background='#364156', foreground='white',
font=('arial',10,'bold'))
    classify_b.place(relx=0.79,rely=0.46)

def upload_image():
    try:
        file_path=filedialog.askopenfilename()
        uploaded=Image.open(file_path)
        uploaded.thumbnail(((top.winfo_width()/2.25),(top.winfo_height()/2.25)))
        im=ImageTk.PhotoImage(uploaded)
        sign_image.configure(image=im)
        sign_image.image=im
        label.configure(text='')
        show_classify_button(file_path)
    except:
        pass

upload=Button(top,text="Upload an image",command=upload_image,padx=10,pady=5)
upload.configure(background='#364156', foreground='white',font=('arial',10,'bold'))
upload.pack(side=BOTTOM,pady=50)
sign_image.pack(side=BOTTOM,expand=True)
label.pack(side=BOTTOM,expand=True)
heading = Label(top, text="Cats&Dogs Classification",pady=20, font=('arial',20,'bold'))
heading.configure(background='#CDCDCD',foreground='#364156')
heading.pack()

top.mainloop()